# Разведывательный анализ

In [1]:
import pandas as pd
from catboost.datasets import titanic

## Основная информация.

In [2]:
titanic_train, titanic_test = titanic()
print(titanic_train.shape)
print(titanic_test.shape)

(891, 12)
(418, 11)


In [3]:
titanic_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Выводы:
* Пропуски есть в трёх признаках - `Age`, `Cabin`, `Embarked`.
* Необходима обработка пропусков для дальнейшей работы.

## Пропуски в признаке `Cabin` личный номер .

In [6]:
titanic_train.dropna()['Cabin'].value_counts()

Cabin
G6             4
B96 B98        4
C23 C25 C27    4
F33            3
D              3
              ..
C91            1
C124           1
C32            1
E34            1
C148           1
Name: count, Length: 133, dtype: int64

In [7]:
titanic_train[titanic_train['Cabin'].isna()]["Pclass"].value_counts()

Pclass
3    479
2    168
1     40
Name: count, dtype: int64

In [8]:
titanic_train["Pclass"].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

Выводы:
* Среди людей с пропущенной личным номером `Cabin` большинство пассажиры 3 класса `Pclass` - 479 человека. Скорее всего у этих пассажиров небыло личного номера, поэтому данные NaN стоит заменить на новый класс - `NoCabin`.
* 168 людей 2 класса. Некоторые из них могли не иметь личного номера, у других быть ошибки в данных. Можно проверить частотные характеристики или корреляции, но для этой работы просто присвоим `NoCabin`.
* 40 людей 1 класса не могут не иметь личного номера. Поэтому, это скорее всего ошибки в данных. Можно заменить на класс `UnknownCabin`.
* Из подводных камней - такая замена может привести к сильной зависимости признаков `Pclass` и `Cabin`. Это не очень хорошо, а именно может возникнуть проблемма мультиколлинеарности.

## Пропуски в признаке `Age` возраст.

In [9]:
titanic_train[titanic_train['Age'].isna()]["Sex"].value_counts()

Sex
male      124
female     53
Name: count, dtype: int64

In [10]:
titanic_train["Sex"].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [11]:
titanic_train[titanic_train['Age'].notna()].groupby('Sex')['Age'].median()

Sex
female    27.0
male      29.0
Name: Age, dtype: float64

Выводы:
* На первый взляд закономерности не видно. Для этой работы просто заменим пропуски на медианный возраст.
* Можно попробовать сделать такую замену с разделением по полу, т.к. женщины могут быть немного моложе.

## Пропуски в признаке `Embarked`.

In [12]:
titanic_train[titanic_train['Embarked'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [13]:
titanic_train[titanic_train['Cabin'] == 'B28']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [14]:
titanic_train['Embarked'].dropna().value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [15]:
titanic_train[titanic_train['Survived'] == 1]['Embarked'].value_counts()

Embarked
S    217
C     93
Q     30
Name: count, dtype: int64

Выводы:
* На первый взляд не видна закономерность пропусков.
* Просто заменим пропуски на самое частотное значение `S`.

## Дубликаты

In [17]:
titanic_train.duplicated().value_counts()

False    891
Name: count, dtype: int64

Выводы:
* Дубликатов нет.

## Отбор признаков

In [31]:
from sklearn.preprocessing import OrdinalEncoder
from scipy.stats import kendalltau

In [19]:
encoder = OrdinalEncoder()

In [25]:
encoder.fit(pd.DataFrame(titanic_train["Ticket"]))

OrdinalEncoder()

In [27]:
titanic_train["Ticket"] = encoder.transform(pd.DataFrame(titanic_train["Ticket"]))

In [32]:
tau, pvalue = kendalltau(titanic_train["Ticket"].values, titanic_train["Survived"].values)
print(tau)
print(pvalue)

-0.1335115283870307
1.0974020992837134e-06


Выводы:
* Коэффициент корреляции Кэндалла очень небольшой.
* Очень маленькое значение p-value (вероятность получить такую случайную выборку), что говорит нам что корреляция скорее всего есть.
* Это может быть просто шум. Чтобы отблосить влияние других признаков, стоит воспользоваться методом последовательного отбора признаков.
* Для данной работы мы этого делать не будем. Просто примем, что признак `Ticket` не важен для предсказания и удалим его.